In [1]:
import os
import re
import sys
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_web_page(url):
    # url = sys.argv[1]  # 第1引数からURLを取得する。
    r = requests.get(url)  # URLで指定したWebページを取得する。
    r.encoding = r.apparent_encoding  # バイト列の特徴から推定したエンコーディングを使用する。
#     print(f'encoding: {r.encoding}', file=sys.stderr)  # エンコーディングを標準エラー出力に出力する。
#     print(r.text)  # デコードしたレスポンスボディを標準出力に出力する。
    # f = open('/Users/kanekotakafumi/github/fake_news_detection_research/data/scraping_data/tmp.txt', 'w')
    # f.writelines(r.text)
    # f.close()
    return r

In [2]:
def link_from_infact_scraping(r):
    link_list = []
    soup = BeautifulSoup(r.text, "html.parser")
    for a in soup.select("div[class='loop-post-thumb normal'] > a"):
        url = a.get('href')
        link_list.append(url)
    
    if not soup.select("link[rel='next']"):
        return link_list
    else:
        next_link = soup.select("link[rel='next']")[0].get('href')
        r = get_web_page(next_link)
        return link_list + link_from_infact_scraping(r)

In [95]:
def fake_news_from_infact_scraping(r, text_list):
    soup = BeautifulSoup(r.text, "html.parser")
    text = soup.select("meta[property='og:title']")
    
    print(str(text))
    pattern = '「.*」'

    repatter = re.compile(pattern)
    result = repatter.search(str(text))
    if result is not None:
        result = result.group()
        result = result.replace(r'「', '').replace(r'」', '')
        text_list.append(result)
        
    text = soup.select("div[class='box-red']")
    print(str(text))
    
    if text is not None:
        pattern = '<div.*?>|</div>|<span.*?>|</span>|<a.*?>.*?</a>|<br/>|</?strong>|（.*）|\(.*|\[.*|\]|<p>.*|</p>'
        result = re.sub(pattern, '', str(text))
        result = result.replace('、',' ')
        result_list = [s for s in result.split() if not s.startswith('#') and s != '「……」']
        
        for result in result_list:
            text_list.append(result)
            
    return text_list

In [96]:
r = get_web_page('https://infact.press/tag/covid-19-factchecks/')
link_list = link_from_infact_scraping(r)
fake_twitter_text_list = []
for link in link_list:
    r = get_web_page(link)
    fake_twitter_text_list = fake_news_from_infact_scraping(r, fake_twitter_text_list)

print(fake_twitter_text_list)

[<meta content="[Factcheck] 「米CDC、PCR検査は精度が低いので中止」は誤り 新指針の曲解が拡散" property="og:title"/>]
[<div class="box-red"><strong><span class="mk-red"><span class="big">チェック対象</span></span></strong><br/>
<div class="css-1dbjc4n">
<div class="css-1dbjc4n r-1s2bzr4">
<div class="css-901oao r-18jsvk2 r-1tl8opc r-1blvdjr r-16dba41 r-vrz42v r-bcqeeo r-bnwqim r-qvutc0" dir="auto" id="id__bqsxsf4t2b9" lang="ja"><span class="r-18u37iz">#PCR検査 </span><span class="css-901oao css-16my406 r-1tl8opc r-bcqeeo r-qvutc0"><strong><span style="text-decoration: underline;">米国CDC『PCR検査は精度が低いので中止します』</span></strong> このウルトラ大ニュースを未だに日本のマスコミはこれを大きく報道しない(笑) ワイドショーであんだけ『PCR検査が必要だ！もっとやれ！』って言ってたのに、どうすんだコレ？ マスコミはコロナ収束させる気ある？</span></div>
</div>
</div>
（Twitter一般ユーザー、2021年8月6日投稿。強調は筆者による）</div>]
[<meta content="[FactCheck]「スウェーデン首相がコロナ禍初期に感染拡大を止められなかった責任を取って辞任」は根拠不明　" property="og:title"/>]
[]
[<meta content="[Fact Check] 「日本の水道水を浴びたコロナ、一瞬で死滅」は根拠不明" property="og:title"/>]
[<div class="box-red"><strong><span class="mk-red"><spa

In [15]:
text_list = []

In [88]:
r = get_web_page(link_list[10])
soup = BeautifulSoup(r.text, "html.parser")
text = soup.select("meta[property='og:title']")

print(str(text))
pattern = '「.*」'

repatter = re.compile(pattern)
result = repatter.search(str(text))
if result is not None:
    result = result.group()
    result = result.replace(r'「', '').replace(r'」', '')
    text_list.append(result)

text = soup.select("div[class='box-red']")

pattern = '<div.*?>|</div>|<span.*?>|</span>|<a.*?>.*?</a>|<br/>|</?strong>|（.*）|\(.*|\[.*|\]'
html = re.sub(pattern, '', str(text))
html = html.replace('、',' ')
[s for s in html.split() if not s.startswith('#')]

[<meta content="[コロナの時代]ファクトチェック： 大阪の重症者増の理由は｢早めに気管切開し､人工呼吸器をつけている｣？ 吉村知事発言は根拠不明" property="og:title"/>]


['治療的な観点でいくと',
 '僕が報告受けているのが',
 '大阪の場合は',
 '死者をできるだけ減らしたいということで',
 'できるだけ早めに気管切開をして',
 '人工呼吸器をつけて',
 'そして命を救う治療を優先している。']

In [83]:
text = soup.select("div[class='box-red']")

pattern = '<div.*?>|</div>|<span.*?>|</span>|<a.*?>.*?</a>|<br/>|</?strong>|（.*）'
html = re.sub(pattern, ' ', str(text))
html.split()

['[',
 'チェック対象',
 '#PCR検査',
 '米国CDC『PCR検査は精度が低いので中止します』',
 'このウルトラ大ニュースを未だに日本のマスコミはこれを大きく報道しない(笑)',
 'ワイドショーであんだけ『PCR検査が必要だ！もっとやれ！』って言ってたのに、どうすんだコレ？',
 'マスコミはコロナ収束させる気ある？',
 ']']

In [62]:
html = re.sub(pattern, ' ', str(text))
html.split()

['[',
 'チェック対象',
 '#PCR検査',
 '米国CDC『PCR検査は精度が低いので中止します』',
 'このウルトラ大ニュースを未だに日本のマスコミはこれを大きく報道しない(笑)',
 'ワイドショーであんだけ『PCR検査が必要だ！もっとやれ！』って言ってたのに、どうすんだコレ？',
 'マスコミはコロナ収束させる気ある？',
 ']']

In [43]:
text = soup.select("div[class='box-red']")
# subでaを除き
html = re.sub('<div.*?>|</a>', '', str(text))
# 正規は簡単にできる
pattern = '<li.*?>(.*?)</li>'

# findall 
results = re.findall(pattern, html, re.S)
# 歌名を出力
for result in results:
    print(result.strip())

[<div class="box-red"><strong><span class="mk-red"><span class="big">チェック対象</span></span></strong><br/><strong>日本の水道水を浴びたコロナ、一瞬で死滅、感染拡大地域はミネラルウォーターが原因か </strong><br/>（5ちゃんねる、2021年5月2日投稿<a href="https://hayabusa9.5ch.net/test/read.cgi/news/1619959618/">スレッド</a>タイトル）</div>]

In [ ]:
box-red

In [21]:
r.text

'<!DOCTYPE html>\n<html lang="ja"><head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#"><meta charset="utf-8" /><meta name="viewport" content="width=device-width,initial-scale=1,minimum-scale=1,user-scalable=yes" /><meta name="description" content="新型コロナウイルス感染症の診断法として、現在日本を始め世界中で広く使われているPCR検査法。そのPCR検査について、アメリカの疾病対策センター（CDC）が中止を決定したという主張がネット上で拡散された。事実はどうなのか、検証した。（大船怜） チェック対象 #PCR検査 米国CDC『PCR検査は精度が低いので中止します』 このウルトラ大ニュースを未だに日本のマスコミはこれを大きく報道しない(笑) ワイドショーであんだけ『PCR検査が必要だ..." /><meta name="keywords" content="ファクトチェック,大船怜,新型コロナウイルス,新型コロナFactCheck" /><meta property="og:title" content="[Factcheck] 「米CDC、PCR検査は精度が低いので中止」は誤り 新指針の曲解が拡散" /><meta property="og:type" content="article" /><meta property="og:url" content="https://infact.press/2021/08/post-13117/" /><meta property="og:image" content="https://infact.press/wp-content/uploads/2021/08/0.png" /><meta property="og:image:width" content="598" /><meta property="og:image:height" content="575" /><meta proper